In [11]:
# Add a check if we run in google colab or locally in jupyter notebook
run_in_colab = False
if 'google.colab' in str(get_ipython()):
    run_in_colab = True
    print('Running on Colab')
else:
    print('Running locally on Jupyter')

Running on Colab


# New Section

In [12]:
# Mount drive in google colab
if run_in_colab:
    from google.colab import drive
    drive.mount('/content/drive')
else:  # Set local path 
    data_path = "/Users/elkysandor/Desktop/huji yr3/static lab/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
if run_in_colab:
    from google.colab import files
    uploaded = files.upload()

Saving nba_data _21.csv to nba_data _21 (2).csv
Saving nba_data_20.csv to nba_data_20 (1).csv


In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import numpy as np  # a module for working with numerical array 
import pandas as pd  # a module for working with data-frames
import statsmodels.api as sm  # a module for statistical modelling (e.g. regression analysis)
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from scipy.optimize import nnls
import seaborn as sns
import datetime
import scipy
import time
import sys
import io

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [86]:
def date_to_datetime(df):
  df.rename(columns={'game date': 'date'},inplace=True)
  df.date = pd.to_datetime(df.date)

def design_data_frame(df):
  to_drop = ["team name","game visitor_team_score","team division", "game_id",
             "minute_in_game", "player_fouls", "game home_team_id", "game home_team_score",
             "game period","game postseason","game season","game status","game time",
             "game visitor_team_id","player height_feet","player height_inches","player team_id",
             "player height_feet","player height_inches","player team_id","player weight_pounds",
             "team id","team full_name","team city","team abbreviation","team conference","player"
             ,"game id","player_defensive_rebounds","player_offensive_rebounds","player_field_three_pointer_percentage",
             "player_field_three_pointers_attempted","player_free_throw_percentage","player_field_goals_percentage",
             "player first_name","player last_name"]
  df["name"] = df["player first_name"].astype(str) + " " + df_2021_nba["player last_name"].astype(str)
  df.drop(columns = to_drop,inplace=True)  


def calculate_data(df):
  to_filter = ["date", "player id", "fantasy_week"]
  numeric_buckets = df.loc[:,~(df.columns.isin(to_filter))]
  agg_df = numeric_buckets.groupby(by = ["week_bucket","name"]).agg(sum)
  agg_df["fg_perc"] = agg_df[["player_field_goal_attempts","player_field_goal_made"]].apply(lambda x : (x.player_field_goal_made/x.player_field_goal_attempts)*100 if x.player_field_goal_attempts != 0 else 0,axis=1)
  agg_df["ft_perc"] = agg_df[["player_free_throws_attempted","player_free_throws_made"]].apply(lambda x : (x.player_free_throws_made/x.player_free_throws_attempted)*100 if x.player_free_throws_attempted != 0 else 0,axis=1)
  # agg_df.drop(columns=["player_field_goal_attempts", "player_free_throws_attempted"],inplace=True) 
  return(agg_df)

def cdf_per_col(col):
  # print(f" {col.loc[1,:]} and {col.index}")
  func_col = col.groupby(level=0).agg(ECDF)
  # print(func_col.index)
  return pd.Series([func_col.loc[i](col.loc[i,:]).round(4) for i in range(1,24)])


def precentages_dat(df,cdf_df):
  for i,j in [("fg_perc","player_field_goal_attempts"),("ft_perc","player_free_throws_attempted")]:
    perc_rating = cdf_df[i] * df[j].values
    avg =(perc_rating.loc[df[j]!=0]).median()
    perc_rating.loc[df[j]==0] = avg
    col_name = '_'.join(j.split("_")[1:3]) + "_rating"
    df[col_name] = perc_rating
  return df

def normalized_df(df,drop_normalized=None):
  to_normalize = df.groupby(level=0).max()
  df.turnover = to_normalize.turnover-df.turnover
  normalized = df.div(to_normalize,level = 0)
  normalized = normalized*100
  wnated_norm = normalized.drop(columns = drop_normalized)
  return(wnated_norm)

def by_time_rating(df,period : str):
  df_mean = df.mean(axis=1)
  if period == "W" or period == "w":
    final = df_mean.groupby(level=[0,1]).agg(np.mean).sort_values(ascending=False)
  elif  period == "Y" or period == "y":  
    final = df_mean.groupby(level=1).agg(np.mean).sort_values(ascending=False)
  else :
    print("please choose y for year or w for week")
    raise ValueError()
  print(df_mean)    
  return(final)


def pipeline(df):
  agg_stat = calculate_data(df_2021_nba)
  perc_cdf = agg_stat[["fg_perc","ft_perc"]].apply(cdf_per_col,axis=0)
  perc_cdf_expand = perc_cdf.explode(list(perc_cdf.columns)).set_index(agg_stat.index.get_level_values(1),append=True)
  new_idx = [(i+1,name)for i,name in perc_cdf_expand.index]
  perc_cdf_expand.index = pd.MultiIndex.from_tuples(new_idx)
  final_agg = precentages_dat(agg_stat,perc_cdf_expand)
  norm_final_agg = normalized_df(final_agg, ["fg_perc","player_field_goal_attempts","ft_perc","player_free_throws_attempted","player_field_goal_made","player_free_throws_made"])
  final_rating = by_time_rating(norm_final_agg,"y")

In [87]:
df_2021_nba = pd.read_csv(io.BytesIO(uploaded['nba_data _21.csv']),  encoding = 'iso-8859-8')
df_2020_nba = pd.read_csv(io.BytesIO(uploaded['nba_data_20.csv']),  encoding = 'iso-8859-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [88]:
date_to_datetime(df_2021_nba)
date_to_datetime(df_2020_nba)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


0        2.641667e+01
1        5.166667e+00
2        2.150000e+01
3        1.808333e+01
4        2.858333e+01
             ...     
29933    5.557008e+04
29934    5.554225e+04
29935    1.491314e+06
29936    3.708333e+01
29937    1.683333e+01
Length: 29938, dtype: float64

In [36]:
# dates for 2021 - 23 weeks
week1 = pd.interval_range(start=pd.Timestamp("2021-10-18"),end=pd.Timestamp("2021-10-24"),periods=1)
before_all_star = pd.interval_range(start=pd.Timestamp("2021-10-24"),end=pd.Timestamp("2022-02-13"),freq="7D")
all_star_week = pd.interval_range(start=pd.Timestamp("2022-02-13"),end=pd.Timestamp("2022-02-27"),periods=1)
after_all_star = pd.interval_range(start=pd.Timestamp("2022-02-27"),end=pd.Timestamp("2022-04-03"),freq="7D")
all_intervals = pd.IntervalIndex([interval for weaks in [week1,before_all_star,all_star_week,after_all_star] for interval in weaks])

In [ ]:
# dates for 2020 - 19 weeks
week_1_20 = pd.interval_range(start=pd.Timestamp("2020-12-21"),end=pd.Timestamp("2020-12-27"),periods=1)
before_all_star_20 = pd.interval_range(start=pd.Timestamp("2020-12-27"),end=pd.Timestamp("2021-02-28"),freq="7D")
all_star_20 = pd.interval_range(start=pd.Timestamp("2021-02-28"),end=pd.Timestamp("2021-03-14"),periods=1)
after_all_star_20 = pd.interval_range(start=pd.Timestamp("2021-03-14"),end=pd.Timestamp("2021-05-09"),freq="7D")
all_intervals_20 = pd.IntervalIndex([interval for weaks in [week_1_20,before_all_star_20,all_star_20,after_all_star_20] for interval in weaks])

In [37]:
df_2021_nba["fantasy_week"] = pd.cut(df_2021_nba.date,all_intervals)
bucket_dict = {week:val+1 for val,week in enumerate(df_2021_nba.fantasy_week.unique())}
df_2021_nba["week_bucket"] = df_2021_nba.fantasy_week.map(bucket_dict)
df_2021_nba.dropna(subset=["fantasy_week"],inplace=True)
# df_2020_nba["fantasy_week"] = pd.cut(df_2020_nba.date,all_intervals_20)
# bucket_dict_20 = {week:val+1 for val,week in enumerate(df_2020_nba.fantasy_week.unique())}
# df_2020_nba["week_bucket"] = df_2020_nba.fantasy_week.map(bucket_dict_20)
# df_2020_nba.dropna(subset=["fantasy_week"],inplace=True)

In [41]:
design_data_frame(df_2021_nba)
# design_data_frame(df_2020_nba)

KeyError: ignored

In [ ]:
guards = df_2021_nba[(df_2021_nba["player position"] != "c") & (df_2021_nba["player position"] != "F-C") & (df_2021_nba["player position"] != "F") & (df_2021_nba["player position"] != "C") & (df_2021_nba["player position"] != "C-F") & (df_2021_nba["player position"].notnull())]
forwards = df_2021_nba[(df_2021_nba["player position"] != "G") & (df_2021_nba["player position"] != "C") & (df_2021_nba["player position"].notnull())]
centers = df_2021_nba[(df_2021_nba["player position"] != "G") & (df_2021_nba["player position"] != "F") & (df_2021_nba["player position"] != "F-G") & (df_2021_nba["player position"] != "G-F") & (df_2021_nba["player position"].notnull())]

In [ ]:
agg_stat = calculate_data(df_2021_nba)
perc_cdf = agg_stat[["fg_perc","ft_perc"]].apply(cdf_per_col,axis=0)
perc_cdf_expand = perc_cdf.explode(list(perc_cdf.columns)).set_index(agg_stat.index.get_level_values(1),append=True)
new_idx = [(i+1,name)for i,name in perc_cdf_expand.index]
perc_cdf_expand.index = pd.MultiIndex.from_tuples(new_idx)
final_agg = precentages_dat(agg_stat,perc_cdf_expand)
norm_final_agg = normalized_df(final_agg, ["fg_perc","player_field_goal_attempts","ft_perc","player_free_throws_attempted","player_field_goal_made","player_free_throws_made"])
final_rating = by_time_rating(norm_final_agg,"y")

# agg_stat_20 = calculate_data(df_2020_nba)
# agg_stat_guards = calculate_data(guards)
# agg_stat_forwards = calculate_data(forwards)
# agg_stat_centers = calculate_data(centers)

In [96]:
(perc_cdf_expand.loc[11]).min()

fg_perc     0.374
ft_perc    0.5345
dtype: object

In [101]:
agg_stat.fg_perc.loc[11].quantile(0.65)

44.0

In [60]:
by_time_rating(norm_final_agg,"w")

week_bucket  name           
1            Aaron Gordon       20.564026
             Aaron Henry        13.392478
             Aaron Holiday      17.217339
             Aaron Nesmith      16.805788
             Aaron Wiggins      15.780053
                                  ...    
23           Zeke Nnaji         14.228962
             Ziaire Williams    23.136981
             Zion Williamson    14.228962
             Zylan Cheatham     14.228962
             nan nan            14.228962
Length: 14329, dtype: float64


week_bucket  name                   
11           LeBron James               79.022003
5            Jayson Tatum               68.168758
4            Stephen Curry              67.521776
10           Shai Gilgeous-Alexander    67.108268
1            Julius Randle              66.103224
                                          ...    
17           Romeo Langford             11.199532
19           Kyle Guy                   11.197660
18           Skylar Mays                11.151873
16           Davis Bertans              11.113897
7            Taj Gibson                 10.974549
Length: 14329, dtype: float64

In [ ]:
# drop_normalized = ["fg_perc","ft_perc","player_field_goal_made","player_free_throws_made"]
# to_normalize = agg_stat.groupby(level=0).max()
# agg_stat.turnover = to_normalize.turnover-agg_stat.turnover
# normalized = agg_stat.div(to_normalize,level = 0)
# normalized = normalized*100
# normalized["field_goal"] = (normalized["fg_perc"] + normalized["player_field_goal_made"])/2
# normalized["free_throw"] = (normalized["ft_perc"] + normalized["player_free_throws_made"])/2
# normalized = normalized.drop(columns = drop_normalized)
# week_rating = normalized.mean(axis=1)
# free_throw = normalized.mean(level = 1)
# free_throw.free_throw.nlargest(10)


NameError: ignored

In [93]:
norm_final_agg.loc[11,"Nikola Jokic"]

player_assists                           50.0
player_blocks                       27.272727
player_field_three_pointers_made     4.761905
player_points                       56.521739
player_rebounds                     69.230769
player_steals                            40.0
turnover                                 40.0
field_goal_rating                   58.724596
free_throws_rating                  69.666348
Name: (11, Nikola Jokic), dtype: object

In [ ]:
week_rating_overall = normalized_df(agg_stat)
week_rating_overall_20 = normalized_df(agg_stat_20)
week_rating_guards = normalized_df(agg_stat_guards)
week_rating_forwards = normalized_df(agg_stat_forwards)
week_rating_centers = normalized_df(agg_stat_centers)

In [ ]:
week_rating_overall

week_bucket  name           
1            Aaron Gordon       22.485952
             Aaron Henry         0.000000
             Aaron Holiday      15.168482
             Aaron Nesmith       3.855755
             Aaron Wiggins       0.000000
                                  ...    
23           Zeke Nnaji          0.000000
             Ziaire Williams    19.134045
             Zion Williamson     0.000000
             Zylan Cheatham      0.000000
             nan nan             0.000000
Length: 14329, dtype: float64

In [ ]:
naive_rating_overall = yearly_rating(week_rating_overall)
naive_rating_overall_20 = yearly_rating(week_rating_overall_20)
naive_rating_guards = yearly_rating(week_rating_guards)
naive_rating_forwards = yearly_rating(week_rating_forwards)
naive_rating_centers = yearly_rating(week_rating_centers)

In [ ]:
naive_rating_guards.nlargest(10)

name
Dejounte Murray      49.144536
Trae Young           47.831503
LaMelo Ball          47.813680
Luka Doncic          47.703421
James Harden         46.911804
Stephen Curry        46.224268
Tyrese Haliburton    46.060438
DeMar DeRozan        45.551952
Fred VanVleet        45.319963
Anthony Edwards      44.957318
dtype: float64

In [ ]:
# display(naive_rating.to_string())
# game_sim
naive_rating_overall_20.nlargest(20)

name
Nikola Jokic             51.577663
Nikola Vucevic           47.727391
Stephen Curry            47.720752
Damian Lillard           46.835031
Luka Doncic              45.373436
Julius Randle            44.734123
Jayson Tatum             43.881674
Giannis Antetokounmpo    43.747874
Rudy Gobert              43.190608
Russell Westbrook        42.648881
Chris Paul               42.579054
Bradley Beal             42.396376
Terry Rozier             42.153590
Kyrie Irving             40.862995
Khris Middleton          40.708104
Trae Young               40.424762
Buddy Hield              40.335912
Joel Embiid              40.284732
Bam Adebayo              40.238458
Fred VanVleet            40.104361
dtype: float64

single player simulation

In [ ]:
# game_sim = agg_stat.drop(columns = ["player_field_goal_made", "player_free_throws_made"])
# game_sim.reset_index(level='name', inplace=True)
# cartesian_product_game_sim = game_sim.assign(key=1).merge(game_sim.assign(key=1), how='left', on = ['key', 'week_bucket'])
# cartesian_product_game_sim = cartesian_product_game_sim.loc[cartesian_product_game_sim["name_x"] != cartesian_product_game_sim["name_y"]]

In [ ]:
# cartesian_product_game_sim

In [ ]:
# def individual_fantasy(df):
#   df["assists_x_won"] = df[["player_assists_x","player_assists_y"]].apply(lambda x : 1 if x.player_assists_x > x.player_assists_y else 0 if x.player_assists_x < x.player_assists_y else 0.5,axis=1)
#   df["blocks_x_won"] = df[["player_blocks_x","player_blocks_y"]].apply(lambda x : 1 if x.player_blocks_x > x.player_blocks_y else 0 if x.player_blocks_x < x.player_blocks_y else 0.5,axis=1)
#   df["three_points_x_won"] = df[["player_field_three_pointers_made_x","player_field_three_pointers_made_y"]].apply(lambda x : 1 if x.player_field_three_pointers_made_x > x.player_field_three_pointers_made_y else 0 if x.player_field_three_pointers_made_x < x.player_field_three_pointers_made_y else 0.5,axis=1)
#   df["points_x_won"] = df[["player_points_x","player_points_y"]].apply(lambda x : 1 if x.player_points_x > x.player_points_y else 0 if x.player_points_x < x.player_points_y else 0.5,axis=1)
#   df["rebounds_x_won"] = df[["player_rebounds_x","player_rebounds_y"]].apply(lambda x : 1 if x.player_rebounds_x > x.player_rebounds_y else 0 if x.player_rebounds_x < x.player_rebounds_y else 0.5,axis=1)
#   df["steals_x_won"] = df[["player_steals_x","player_steals_y"]].apply(lambda x : 1 if x.player_steals_x > x.player_steals_y else 0 if x.player_steals_x < x.player_steals_y else 0.5,axis=1)
#   df["turnover_x_won"] = df[["turnover_x","turnover_y"]].apply(lambda x : 1 if x.turnover_x < x.turnover_y else 0 if x.turnover_x > x.turnover_y else 0.5,axis=1)
#   df["fg_perc_x_won"] = df[["fg_perc_x","fg_perc_y"]].apply(lambda x : 1 if x.fg_perc_x > x.fg_perc_y else 0 if x.fg_perc_x < x.fg_perc_y else 0.5,axis=1)
#   df["ft_x_won"] = df[["ft_perc_x","ft_perc_y"]].apply(lambda x : 1 if x.ft_perc_x > x.ft_perc_y else 0 if x.ft_perc_x < x.ft_perc_y else 0.5,axis=1)
#   return df

In [ ]:
# gili_sim = individual_fantasy(cartesian_product_game_sim)

In [ ]:
# gil_sim = gili_sim.get(['name_x','assists_x_won','blocks_x_won', 'three_points_x_won','points_x_won','rebounds_x_won','steals_x_won','turnover_x_won','fg_perc_x_won','ft_x_won'])
# gil_sim.set_index('name_x',inplace=True)
# gil_sum = gil_sim.sum(axis=1)
# individual_fantasy_final = gil_sum.groupby(level =0).agg(np.sum).sort_values(ascending=False)
# individual_fantasy_final.nlargest(20)

In [ ]:
elky_mind = ECDF(agg_stat.player_assists)
len(elky_mind(agg_stat.player_assists))

14329

In [ ]:
ecdf_df = agg_stat.apply(cdf_per_col,axis=0)
ecdf_nba = ecdf_df.explode(list(ecdf_df.columns)).set_index(agg_stat.index.get_level_values(1),append=True)
ecdf_nba = ecdf_nba.astype(float)
# ecdf_nba["player_free_throws_made_weight"] = pd.cut(ecdf_nba.player_free_throws_made,4)
# ecdf_nba["free_throw"] = 
groupby_ft = (ecdf_nba.player_free_throws_made.groupby(level=0)).quantile([0.25,.5,.75,1])
# md = pd.qcut(groupby_ft,4,labels=[0,0.3,0.65,1]).astype(float)
# fg = pd.qcut(groupby_ft,4,labels=[0,0.3,0.65,1]).astype(int)
# ecdf_nba["free_throw"] = ecdf_nba.ft_perc*0.7 + md*0.3
# ecdf_nba["field_goal"] = ecdf_nba.fg_perc*0.7 +fg*0.3
groupby_ft

0   0.25    0.6132
    0.50    0.6132
    0.75    0.7592
    1.00    1.0000
1   0.25    0.5441
             ...  
21  1.00    1.0000
22  0.25    0.5409
    0.50    0.5409
    0.75    0.7817
    1.00    1.0000
Name: player_free_throws_made, Length: 92, dtype: float64

In [ ]:
ecdf_nba.astype(np.float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


player_assists  player_blocks  \
   name                                             
0  Aaron Gordon             0.8106         0.8411   
   Aaron Henry              0.4896         0.7063   
   Aaron Holiday            0.6469         0.9117   
   Aaron Nesmith            0.6982         0.7063   
   Aaron Wiggins            0.4896         0.7063   
...                            ...            ...   
22 Zeke Nnaji               0.4687         0.6404   
   Ziaire Williams          0.7769         0.7961   
   Zion Williamson          0.4687         0.6404   
   Zylan Cheatham           0.4687         0.6404   
   nan nan                  0.4687         0.6404   

                    player_field_three_pointers_made  player_field_goal_made  \
   name                                                                        
0  Aaron Gordon                               0.6870                  0.7512   
   Aaron Henry                                0.5859                  0.4494   
   Aaron Holiday                              0.6870                  0.5522   
   Aaron Nesmith                              0.5859                  0.4494   
   Aaron Wiggins                              0.5859                  0.4494   
...                                              ...                     ...   
22 Zeke Nnaji                                 0.5506                  0.4270   
   Ziaire Williams                            0.6982                  0.7175   
   Zion Williamson                            0.5506                  0.4270   
   Zylan Cheatham                             0.5506                  0.4270   
   nan nan                                    0.5506                  0.4270   

                    player_free_throws_made  player_points  player_rebounds  \
   name                                                                       
0  Aaron Gordon                      0.7994         0.7335           0.8395   
   Aaron Henry                       0.6132         0.4350           0.4398   
   Aaron Holiday                     0.7592         0.5538           0.6276   
   Aaron Nesmith                     0.6132         0.4350           0.5827   
   Aaron Wiggins                     0.6132         0.4350           0.4398   
...                                     ...            ...              ...   
22 Zeke Nnaji                        0.5409         0.4270           0.4189   
   Ziaire Williams                   0.8668         0.7255           0.6758   
   Zion Williamson                   0.5409         0.4270           0.4189   
   Zylan Cheatham                    0.5409         0.4270           0.4189   
   nan nan                           0.5409         0.4270           0.4189   

                    player_steals  turnover  fg_perc  ft_perc  
   name                                                        
0  Aaron Gordon            0.5698    0.9246   0.9470   0.7945  
   Aaron Henry             0.5698    0.5072   0.4494   0.6132  
   Aaron Holiday           0.5698    0.8010   0.4671   1.0000  
   Aaron Nesmith           0.7287    0.6356   0.4494   0.6132  
   Aaron Wiggins           0.5698    0.5072   0.4494   0.6132  
...                           ...       ...      ...      ...  
22 Zeke Nnaji              0.5281    0.4880   0.4270   0.5409  
   Ziaire Williams         0.7624    0.6790   0.5939   0.8668  
   Zion Williamson         0.5281    0.4880   0.4270   0.5409  
   Zylan Cheatham          0.5281    0.4880   0.4270   0.5409  
   nan nan                 0.5281    0.4880   0.4270   0.5409  

[14329 rows x 11 columns]

In [ ]:
ecdf = ECDF([1, 2, 3, 4])
ecdf([1,2,3,4])

array([0.25, 0.5 , 0.75, 1.  ])